In [7]:
import pandas as pd
import numpy as np
import MAIN.Basics as basics
import MAIN.Reinforcement as RL
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from UTIL import FileIO
from STRATEGY.Cointegration import EGCointegration
import time
import warnings
warnings.filterwarnings('ignore')



In [8]:
# Read config
config_path  = 'CONFIG/config_train.yml'
config_train = FileIO.read_yaml(config_path)

# Read prices
# x_stock = 'PEP'
# y_stock = 'KO'
# x_stock = 'JNJ'
# y_stock = 'PG'
# x_stock = 'GOOG'
# y_stock = 'FB'
# x_stock = 'C'
# y_stock = 'WFC'
x_stock = 'C'
y_stock = 'BK'
x = pd.read_csv('STATICS/PRICE/'+ x_stock +'.csv')
y = pd.read_csv('STATICS/PRICE/'+ y_stock +'.csv')
x, y = EGCointegration.clean_data(x, y, 'date', 'close')


In [9]:
def train_model(x,y): 
    # Separate training and testing sets
    train_pct = 0.7
    train_len = round(len(x) * 0.7)
    idx_train = list(range(0, train_len))
    idx_test  = list(range(train_len, len(x)))
    EG_Train = EGCointegration(x.iloc[idx_train, :], y.iloc[idx_train, :], 'date', 'close')
    EG_Test  = EGCointegration(x.iloc[idx_test,  :], y.iloc[idx_test,  :], 'date', 'close')

    # Create action space
    n_hist    = list(np.arange(60, 601, 60))
    n_forward = list(np.arange(120, 1201, 120))
    trade_th  = list(np.arange(1,  5.1, 1))
    stop_loss = list(np.arange(1,  2.1, 0.5))
    cl        = list(np.arange(0.05,  0.11, 0.05))
    actions   = {'n_hist':    n_hist,
                 'n_forward': n_forward,
                 'trade_th':  trade_th,
                 'stop_loss': stop_loss,
                 'cl':        cl}
    n_action  = int(np.product([len(actions[key]) for key in actions.keys()]))

    # Create state space
    transaction_cost = [0.001]
    states  = {'transaction_cost': transaction_cost}
    n_state = len(states)

    # Assign state and action spaces to config
    config_train['StateSpaceState'] = states
    config_train['ActionSpaceAction'] = actions

    # Create and build network
    one_hot  = {'one_hot': {'func_name':  'one_hot',
                            'input_arg':  'indices',
                             'layer_para': {'indices': None,
                                            'depth': n_state}}}
    output_layer = {'final': {'func_name':  'fully_connected',
                              'input_arg':  'inputs',
                              'layer_para': {'inputs': None,
                                             'num_outputs': n_action,
                                             'biases_initializer': None,
                                             'activation_fn': tf.nn.relu,
                                             'weights_initializer': tf.ones_initializer()}}}

    state_in = tf.placeholder(shape=[1], dtype=tf.int32)

    N = basics.Network(state_in)
    N.build_layers(one_hot)
    N.add_layer_duplicates(output_layer, 1)

    # Create learning object and perform training
    RL_Train = RL.ContextualBandit(N, config_train, EG_Train)

    start = time.time()
    sess = tf.Session()
    RL_Train.process(sess, save=False, restore=False)
    end = time.time()

    # Extract training results
    action = RL_Train.recorder.record['NETWORK_ACTION']
    reward = RL_Train.recorder.record['ENGINE_REWARD']
    [opt_action] = sess.run([RL_Train.output], feed_dict=RL_Train.feed_dict)
    opt_action = np.argmax(opt_action)
    action_dict = RL_Train.action_space.convert(opt_action, 'index_to_dict')

    return action, reward, action_dict, (end-start)/60

In [ ]:
df_result = pd.DataFrame()

for i in range(5):
    action, reward, action_dict, time_taken = train_model(x,y)

    df1 = pd.DataFrame()
    df1['action'] = action
    df1['reward'] = reward
    mean_reward_per_action = df1.groupby('action').mean()
    #sns.distplot(mean_reward_per_action)


    print(mean_reward_per_action.reward.mean())
    print(f"Training took: {time_taken}")

    df_result.loc[i,'action_dict'] = str(action_dict)
    df_result.loc[i,'mean_reward'] = mean_reward_per_action.reward.mean()
    df_result.loc[i,'time_taken'] = time_taken
    df_result.to_csv('df_result_30.csv', index=False)


In [ ]:
df_result

In [ ]:
print("a")